In [11]:
import os
import glob
import numpy as np
import medleydb as mdb
from medleydb import mix
import sox
import shutil

## Make MedleyDB training data

In [7]:
MDB_AUDIO_DIR = '/Users/rachelbittner/Documents/VocalAnnotations_MDB/audio/'
MDB_ANNOT_DIR = '/Users/rachelbittner/Documents/VocalAnnotations_MDB/annotations'

In [23]:
mtracks = mdb.load_melody_multitracks()
for mtrack in mtracks:
    print(mtrack.track_id)

    if mtrack.predominant_stem.instrument not in mix.VOCALS:
        print "no vocals...skipping"
        print ""
        continue

    output_path = os.path.join(MDB_AUDIO_DIR, '{}_vocal.wav'.format(mtrack.track_id))
    print([(i, v.instrument) for i, v in mtrack.stems.items()])
        
    need_remix = False
    stem_indices = mtrack.stems.keys()
    stem_indices.remove(mtrack.predominant_stem.stem_idx)
    for i in stem_indices:
        if (mtrack.stems[i].instrument in mix.VOCALS):
            need_remix = True

    if need_remix:
        print("needs remix")
        mix_stems = [i for i, v in mtrack.stems.items() if v.instrument not in mix.VOCALS]
        mix_stems.append(mtrack.predominant_stem.stem_idx)
        mix.mix_multitrack(mtrack, output_path, stem_indices=mix_stems)
    else:
        print("using orignal mix")
        shutil.copy(mtrack.mix_path, output_path)

    melody1_fpath = os.path.join(mtrack.annotation_dir, mdb.multitrack._MELODY1_FMT % mtrack.track_id)
    annotation_fpath = os.path.join(MDB_ANNOT_DIR, '{}_vocal.csv'.format(mtrack.track_id))
    shutil.copy(melody1_fpath, annotation_fpath)
    print ""

AClassicEducation_NightOwl
[(1, 'electric bass'), (2, 'drum set'), (3, 'distorted electric guitar'), (4, 'distorted electric guitar'), (5, 'clean electric guitar'), (6, 'distorted electric guitar'), (7, 'distorted electric guitar'), (8, 'male singer'), (9, 'synthesizer'), (10, 'vocalists'), (11, 'tambourine'), (12, 'fx/processed sound'), (13, 'male singer')]
needs remix

AimeeNorwich_Child
[(1, 'drum set'), (2, 'electric bass'), (3, 'clean electric guitar'), (4, 'female singer'), (5, 'vocalists'), (6, 'piano'), (7, 'acoustic guitar')]
needs remix

AimeeNorwich_Flying
no vocals...skipping

AlexanderRoss_GoodbyeBolero
[(1, 'acoustic guitar'), (2, 'clean electric guitar'), (3, 'clean electric guitar'), (4, 'electric bass'), (5, 'drum set'), (6, 'male singer'), (7, 'brass section'), (8, 'viola section'), (9, 'harmonica')]
using orignal mix

AlexanderRoss_VelvetCurtain
[(1, 'acoustic guitar'), (2, 'clean electric guitar'), (3, 'clean electric guitar'), (4, 'electric bass'), (5, 'drum set'),

# Make World Music Validation Set

In [5]:
WORLD_MUSIC_AUDIO_DIR = '/Users/rachelbittner/Documents/VocalAnnotations_Tony/audio/'
WORLD_MUSIC_TONY_DIR = '/Users/rachelbittner/Documents/VocalAnnotations_Tony/tony/'
WORLD_MUSIC_ANNOT_DIR = '/Users/rachelbittner/Documents/VocalAnnotations_Tony/annotations/'
TONY_SCRIPT = '/Users/rachelbittner/Dropbox/MARL/repos/medleydb/scripts/fill_tony_file.py'

In [6]:
for audio_fpath in glob.glob(os.path.join(WORLD_MUSIC_AUDIO_DIR, '*.wav')):
    fname = os.path.basename(audio_fpath).split('.')[0]
    tony_fpath = os.path.join(WORLD_MUSIC_TONY_DIR, "{}.csv".format(fname))
    output_fpath = os.path.join(WORLD_MUSIC_ANNOT_DIR, "{}.csv".format(fname))
    duration = sox.file_info.duration(audio_fpath)
    os.system('python {} {} {} {}'.format(TONY_SCRIPT, tony_fpath, output_fpath, duration))